## Create a notebook to perform Runge-Kutta integration for multiple coupled variables.

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

This cell and the one following it are not a requirement, it is only for looks

In [14]:
#use colors.subclass(or command; e.g bold).colorname to print
#examples: print(colors.bold, colors.fg.blue, "this will be bold and blue")
#everything after this will have that format until the following command
#is given: print(colors.reset, "now, this text will be normal")

class colors:
    reset='\033[0m' #reset all colors with colors.reset
    bold='\033[01m'
    underline='\033[04m'
    strikethrough='\033[09m'
    reverse='\033[07m'
    class fg:  #foreground subclass
        black='\033[30m'
        red='\033[31m'
        green='\033[32m'
        orange='\033[33m'
        blue='\033[34m'
        purple='\033[35m'
        cyan='\033[36m'
        lightgrey='\033[37m'
        darkgrey='\033[90m'
        lightred='\033[91m'
        lightgreen='\033[92m'
        yellow='\033[93m'
        lightblue='\033[94m'
        pink='\033[95m'
        lightcyan='\033[96m'
    class bg: #background subclass
        black='\033[40m'
        red='\033[41m'
        green='\033[42m'
        orange='\033[43m'
        blue='\033[44m'
        purple='\033[45m'
        cyan='\033[46m'
        lightgrey='\033[47m'

The above code was provided by https://www.geeksforgeeks.org/print-colors-python-terminal/

### Define our coupled derivatives to integrate

In [15]:
def dydx(x,y):
    
    #set the derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #so we can write 
    #dydx = z
    #dzdx = -y
    
    #we will set y = y[0]
    #we will set z = y[1]
    
    #declare an array
    y_derivs = np.zeros(2)
    
    #set dydx = z
    y_derivs[0] =    y[1]
    
    #set dzdx = -y
    y_derivs[1] = -1*y[0]
    
    #here we have to return an array with dydx
    return y_derivs

### Define the 4th order RK method

In [16]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    
    #declare k? arrays; (? is a wildcard, used for k1,k2,...,kn)
    k1=np.zeros(nv)
    k2=np.zeros(nv)
    k3=np.zeros(nv)
    k4=np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    x_ipo = xi + h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dydx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipo,y_temp)
    k4[:] = h*y_derivs[:]
    
    
    #advance y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

### Define an adaptive step size driver for RK4

In [17]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY    = 0.9
    H_NEW_FAC = 2.0
    
    #set a max number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #create an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        #estimate our error by taking one step of size h
        #vs. two steps of size h/2
        y_2  = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1  = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        
        #compute an error
        Delta = np.fabs(y_2 - y_11)
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our error is too large, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        #check iteration
        if(i>=imax):
            print("Too many iterations in rk4_mv_ad()")
            raise StopIteration("Ending after i = ",i)
            
        #iterate
        i+=1
        
        
    #next time, try to take a bigger step
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
    
    #return the answer, a new step, and the step we actually took
    return y_2, h_new, h_step

## Define a wrapper for RK4

In [20]:
def rk4_mv(dydx,a,b,y_a,tol):
    
    #dfdx is   derivative w.r.t. x
    #a    is   lower bound
    #b    is   upper bound
    #y_a  are  boundary conditions
    #tol  is   tolerance for integrating y
    
    #define our starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h = 1.0e-4 * (b-a)
    
    #set max number of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set the number of coupled ODEs to the size of y_a
    nv = len(y_a)
    
    
    #set the initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    #loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            #recalculate y_i+1
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            #break
            flag = 0
            
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]
        del y
        y = y_new
        
        #prevent too many iterations
        if(i>=imax):
            
            print(colors.bold, colors.fg.red, colors.bg.black, "Maximum iterations reached.", colors.reset)
            raise StopIteration("Iteration number = ",i)
            
            
        #iterate
        i += 1
        
        #output some information
        s = "i =%3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step,b)
        print(s)
        
        #break if new xi is == b
        if(xi==b):
            flag = 0
            
    #return the answer
    print(colors.bold, colors.fg.purple, "Iteration #", i, colors.reset)
    return x,y

### Perform the integration

In [21]:
a = 0.0
b = 2.0 * np.pi

y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6

#perform the integration
x,y = rk4_mv(dydx,a,b,y_0,tolerance)

i =  1	x = 0.00000186	h = 0.00000186	b=6.28318531
i =  2	x = 0.00000384	h = 0.00000199	b=6.28318531
i =  3	x = 0.00000584	h = 0.00000200	b=6.28318531
i =  4	x = 0.00000784	h = 0.00000200	b=6.28318531
i =  5	x = 0.00000984	h = 0.00000200	b=6.28318531
i =  6	x = 0.00001184	h = 0.00000200	b=6.28318531
i =  7	x = 0.00001384	h = 0.00000200	b=6.28318531
i =  8	x = 0.00001584	h = 0.00000200	b=6.28318531
i =  9	x = 0.00001784	h = 0.00000200	b=6.28318531
i = 10	x = 0.00001984	h = 0.00000200	b=6.28318531
i = 11	x = 0.00002184	h = 0.00000200	b=6.28318531
i = 12	x = 0.00002384	h = 0.00000200	b=6.28318531
i = 13	x = 0.00002584	h = 0.00000200	b=6.28318531
i = 14	x = 0.00002784	h = 0.00000200	b=6.28318531
i = 15	x = 0.00002984	h = 0.00000200	b=6.28318531
i = 16	x = 0.00003184	h = 0.00000200	b=6.28318531
i = 17	x = 0.00003384	h = 0.00000200	b=6.28318531
i = 18	x = 0.00003584	h = 0.00000200	b=6.28318531
i = 19	x = 0.00003784	h = 0.00000200	b=6.28318531
i = 20	x = 0.00003984	h = 0.00000200	b=6.28318531


i =610	x = 0.00121984	h = 0.00000200	b=6.28318531
i =611	x = 0.00122184	h = 0.00000200	b=6.28318531
i =612	x = 0.00122384	h = 0.00000200	b=6.28318531
i =613	x = 0.00122584	h = 0.00000200	b=6.28318531
i =614	x = 0.00122784	h = 0.00000200	b=6.28318531
i =615	x = 0.00122984	h = 0.00000200	b=6.28318531
i =616	x = 0.00123184	h = 0.00000200	b=6.28318531
i =617	x = 0.00123384	h = 0.00000200	b=6.28318531
i =618	x = 0.00123584	h = 0.00000200	b=6.28318531
i =619	x = 0.00123784	h = 0.00000200	b=6.28318531
i =620	x = 0.00123984	h = 0.00000200	b=6.28318531
i =621	x = 0.00124184	h = 0.00000200	b=6.28318531
i =622	x = 0.00124384	h = 0.00000200	b=6.28318531
i =623	x = 0.00124584	h = 0.00000200	b=6.28318531
i =624	x = 0.00124784	h = 0.00000200	b=6.28318531
i =625	x = 0.00124984	h = 0.00000200	b=6.28318531
i =626	x = 0.00125184	h = 0.00000200	b=6.28318531
i =627	x = 0.00125384	h = 0.00000200	b=6.28318531
i =628	x = 0.00125584	h = 0.00000200	b=6.28318531
i =629	x = 0.00125784	h = 0.00000200	b=6.28318531


i =1140	x = 0.00227984	h = 0.00000200	b=6.28318531
i =1141	x = 0.00228184	h = 0.00000200	b=6.28318531
i =1142	x = 0.00228384	h = 0.00000200	b=6.28318531
i =1143	x = 0.00228584	h = 0.00000200	b=6.28318531
i =1144	x = 0.00228784	h = 0.00000200	b=6.28318531
i =1145	x = 0.00228984	h = 0.00000200	b=6.28318531
i =1146	x = 0.00229184	h = 0.00000200	b=6.28318531
i =1147	x = 0.00229384	h = 0.00000200	b=6.28318531
i =1148	x = 0.00229584	h = 0.00000200	b=6.28318531
i =1149	x = 0.00229784	h = 0.00000200	b=6.28318531
i =1150	x = 0.00229984	h = 0.00000200	b=6.28318531
i =1151	x = 0.00230184	h = 0.00000200	b=6.28318531
i =1152	x = 0.00230384	h = 0.00000200	b=6.28318531
i =1153	x = 0.00230584	h = 0.00000200	b=6.28318531
i =1154	x = 0.00230784	h = 0.00000200	b=6.28318531
i =1155	x = 0.00230984	h = 0.00000200	b=6.28318531
i =1156	x = 0.00231184	h = 0.00000200	b=6.28318531
i =1157	x = 0.00231384	h = 0.00000200	b=6.28318531
i =1158	x = 0.00231584	h = 0.00000200	b=6.28318531
i =1159	x = 0.00231784	h = 0.00

i =1645	x = 0.00328985	h = 0.00000200	b=6.28318531
i =1646	x = 0.00329185	h = 0.00000200	b=6.28318531
i =1647	x = 0.00329385	h = 0.00000200	b=6.28318531
i =1648	x = 0.00329585	h = 0.00000200	b=6.28318531
i =1649	x = 0.00329785	h = 0.00000200	b=6.28318531
i =1650	x = 0.00329985	h = 0.00000200	b=6.28318531
i =1651	x = 0.00330185	h = 0.00000200	b=6.28318531
i =1652	x = 0.00330385	h = 0.00000200	b=6.28318531
i =1653	x = 0.00330585	h = 0.00000200	b=6.28318531
i =1654	x = 0.00330785	h = 0.00000200	b=6.28318531
i =1655	x = 0.00330985	h = 0.00000200	b=6.28318531
i =1656	x = 0.00331185	h = 0.00000200	b=6.28318531
i =1657	x = 0.00331385	h = 0.00000200	b=6.28318531
i =1658	x = 0.00331585	h = 0.00000200	b=6.28318531
i =1659	x = 0.00331785	h = 0.00000200	b=6.28318531
i =1660	x = 0.00331985	h = 0.00000200	b=6.28318531
i =1661	x = 0.00332185	h = 0.00000200	b=6.28318531
i =1662	x = 0.00332385	h = 0.00000200	b=6.28318531
i =1663	x = 0.00332585	h = 0.00000200	b=6.28318531
i =1664	x = 0.00332785	h = 0.00

i =2123	x = 0.00424585	h = 0.00000200	b=6.28318531
i =2124	x = 0.00424785	h = 0.00000200	b=6.28318531
i =2125	x = 0.00424985	h = 0.00000200	b=6.28318531
i =2126	x = 0.00425185	h = 0.00000200	b=6.28318531
i =2127	x = 0.00425385	h = 0.00000200	b=6.28318531
i =2128	x = 0.00425585	h = 0.00000200	b=6.28318531
i =2129	x = 0.00425785	h = 0.00000200	b=6.28318531
i =2130	x = 0.00425985	h = 0.00000200	b=6.28318531
i =2131	x = 0.00426185	h = 0.00000200	b=6.28318531
i =2132	x = 0.00426385	h = 0.00000200	b=6.28318531
i =2133	x = 0.00426585	h = 0.00000200	b=6.28318531
i =2134	x = 0.00426785	h = 0.00000200	b=6.28318531
i =2135	x = 0.00426985	h = 0.00000200	b=6.28318531
i =2136	x = 0.00427185	h = 0.00000200	b=6.28318531
i =2137	x = 0.00427385	h = 0.00000200	b=6.28318531
i =2138	x = 0.00427585	h = 0.00000200	b=6.28318531
i =2139	x = 0.00427785	h = 0.00000200	b=6.28318531
i =2140	x = 0.00427985	h = 0.00000200	b=6.28318531
i =2141	x = 0.00428185	h = 0.00000200	b=6.28318531
i =2142	x = 0.00428385	h = 0.00

...

i =8372	x = 0.01674462	h = 0.00000200	b=6.28318531
i =8373	x = 0.01674662	h = 0.00000200	b=6.28318531
i =8374	x = 0.01674862	h = 0.00000200	b=6.28318531
i =8375	x = 0.01675062	h = 0.00000200	b=6.28318531
i =8376	x = 0.01675262	h = 0.00000200	b=6.28318531
i =8377	x = 0.01675462	h = 0.00000200	b=6.28318531
i =8378	x = 0.01675663	h = 0.00000200	b=6.28318531
i =8379	x = 0.01675863	h = 0.00000200	b=6.28318531
i =8380	x = 0.01676063	h = 0.00000200	b=6.28318531
i =8381	x = 0.01676263	h = 0.00000200	b=6.28318531
i =8382	x = 0.01676463	h = 0.00000200	b=6.28318531
i =8383	x = 0.01676663	h = 0.00000200	b=6.28318531
i =8384	x = 0.01676863	h = 0.00000200	b=6.28318531
i =8385	x = 0.01677063	h = 0.00000200	b=6.28318531
i =8386	x = 0.01677263	h = 0.00000200	b=6.28318531
i =8387	x = 0.01677463	h = 0.00000200	b=6.28318531
i =8388	x = 0.01677663	h = 0.00000200	b=6.28318531
i =8389	x = 0.01677863	h = 0.00000200	b=6.28318531
i =8390	x = 0.01678063	h = 0.00000200	b=6.28318531
i =8391	x = 0.01678263	h = 0.00

i =8778	x = 0.01755674	h = 0.00000200	b=6.28318531
i =8779	x = 0.01755874	h = 0.00000200	b=6.28318531
i =8780	x = 0.01756074	h = 0.00000200	b=6.28318531
i =8781	x = 0.01756274	h = 0.00000200	b=6.28318531
i =8782	x = 0.01756474	h = 0.00000200	b=6.28318531
i =8783	x = 0.01756674	h = 0.00000200	b=6.28318531
i =8784	x = 0.01756874	h = 0.00000200	b=6.28318531
i =8785	x = 0.01757074	h = 0.00000200	b=6.28318531
i =8786	x = 0.01757275	h = 0.00000200	b=6.28318531
i =8787	x = 0.01757475	h = 0.00000200	b=6.28318531
i =8788	x = 0.01757675	h = 0.00000200	b=6.28318531
i =8789	x = 0.01757875	h = 0.00000200	b=6.28318531
i =8790	x = 0.01758075	h = 0.00000200	b=6.28318531
i =8791	x = 0.01758275	h = 0.00000200	b=6.28318531
i =8792	x = 0.01758475	h = 0.00000200	b=6.28318531
i =8793	x = 0.01758675	h = 0.00000200	b=6.28318531
i =8794	x = 0.01758875	h = 0.00000200	b=6.28318531
i =8795	x = 0.01759075	h = 0.00000200	b=6.28318531
i =8796	x = 0.01759275	h = 0.00000200	b=6.28318531
i =8797	x = 0.01759475	h = 0.00

i =9195	x = 0.01839088	h = 0.00000200	b=6.28318531
i =9196	x = 0.01839288	h = 0.00000200	b=6.28318531
i =9197	x = 0.01839488	h = 0.00000200	b=6.28318531
i =9198	x = 0.01839688	h = 0.00000200	b=6.28318531
i =9199	x = 0.01839888	h = 0.00000200	b=6.28318531
i =9200	x = 0.01840088	h = 0.00000200	b=6.28318531
i =9201	x = 0.01840288	h = 0.00000200	b=6.28318531
i =9202	x = 0.01840488	h = 0.00000200	b=6.28318531
i =9203	x = 0.01840688	h = 0.00000200	b=6.28318531
i =9204	x = 0.01840888	h = 0.00000200	b=6.28318531
i =9205	x = 0.01841088	h = 0.00000200	b=6.28318531
i =9206	x = 0.01841288	h = 0.00000200	b=6.28318531
i =9207	x = 0.01841488	h = 0.00000200	b=6.28318531
i =9208	x = 0.01841688	h = 0.00000200	b=6.28318531
i =9209	x = 0.01841888	h = 0.00000200	b=6.28318531
i =9210	x = 0.01842088	h = 0.00000200	b=6.28318531
i =9211	x = 0.01842288	h = 0.00000200	b=6.28318531
i =9212	x = 0.01842488	h = 0.00000200	b=6.28318531
i =9213	x = 0.01842688	h = 0.00000200	b=6.28318531
i =9214	x = 0.01842888	h = 0.00

i =9593	x = 0.01918702	h = 0.00000200	b=6.28318531
i =9594	x = 0.01918902	h = 0.00000200	b=6.28318531
i =9595	x = 0.01919102	h = 0.00000200	b=6.28318531
i =9596	x = 0.01919302	h = 0.00000200	b=6.28318531
i =9597	x = 0.01919502	h = 0.00000200	b=6.28318531
i =9598	x = 0.01919702	h = 0.00000200	b=6.28318531
i =9599	x = 0.01919902	h = 0.00000200	b=6.28318531
i =9600	x = 0.01920102	h = 0.00000200	b=6.28318531
i =9601	x = 0.01920302	h = 0.00000200	b=6.28318531
i =9602	x = 0.01920502	h = 0.00000200	b=6.28318531
i =9603	x = 0.01920702	h = 0.00000200	b=6.28318531
i =9604	x = 0.01920902	h = 0.00000200	b=6.28318531
i =9605	x = 0.01921102	h = 0.00000200	b=6.28318531
i =9606	x = 0.01921302	h = 0.00000200	b=6.28318531
i =9607	x = 0.01921502	h = 0.00000200	b=6.28318531
i =9608	x = 0.01921702	h = 0.00000200	b=6.28318531
i =9609	x = 0.01921902	h = 0.00000200	b=6.28318531
i =9610	x = 0.01922102	h = 0.00000200	b=6.28318531
i =9611	x = 0.01922302	h = 0.00000200	b=6.28318531
i =9612	x = 0.01922503	h = 0.00

StopIteration: ('Iteration number = ', 10000)